In [1]:
import networkx as nx
import sys

In [2]:
def load_network(string_link_file, node_info, cutoff=750):
    network = nx.Graph()
    for line in open(string_link_file):
        line = line.rstrip()
        if line.startswith('protein'):
            continue
        else:
            try:
             p1, p2, score = line.split()
            except ValueError:
                print(line)
                sys.exit(1)
                
            score = float(score)
            
            if score >= cutoff:
                p1 = node_info[p1]
                p2 = node_info[p2]
                network.add_edge(p1, p2)
    
    return network

In [3]:
def load_node_info(string_info_file):
    node_info = {}
    for line in open(string_info_file):
        line = line.rstrip() 
        if line.startswith('#'):
            continue
        else:
            items = line.split('\t')
            p = items[0]
            name = items[1]
            
            node_info[p] = name
    
    return node_info

In [4]:
center = 'VHL' # modifying center as your gene

In [5]:
node_info =  load_node_info('9606.protein.info.v11.5.txt')
G = load_network('9606.protein.links.v11.5.txt', node_info, cutoff=750)
    
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G0 = G.subgraph(Gcc[0]) # select the largest connected component
    

first_nbs = [nb for nb in G0.neighbors(center)]
second_nbs = []
    
for n in first_nbs:
    second_nbs = second_nbs + [nb for nb in G0.neighbors(n)]
    
second_nbs_nr = set(second_nbs) - set(first_nbs+[center])
        
G0.nodes[center]['distance'] = 0
    
for nb in first_nbs:
    G0.nodes[nb]['distance'] = 1
        
for nb in second_nbs_nr:
    G0.nodes[nb]['distance'] = 2
    
#nx.write_gml(G0, path='gene_highlighted_network.gml')

sub = G.subgraph([center]+first_nbs+list(second_nbs_nr))
nx.write_gml(sub, path=center+'_centered_subnetwork.gml')
    
f = open(center+'_subnetwork_members.txt', 'w')
for node in sub.nodes():
    f.write(node+'\n')
f.close()